In [ ]:
# Code to resize images to a specified size if required

In [12]:
import os
from PIL import Image

In [27]:
src = "/media/sid/Seagate Expansion Drive/old-recordings/calibration-images/common/calibration_images_common_from_drive/kinect0/stereo-calibration/originals"

# src = '/media/sid/Seagate Expansion Drive/synchronization-images'
subdirs = [x[0] for x in os.walk(src)]
rgb_dirs = [x for x in subdirs if x.endswith('rgb')]
omni_dirs = [x for x in subdirs if x.endswith('omni')]
ir_dirs = [x for x in subdirs if x.endswith('ir')]
depth_dirs = [x for x in subdirs if x.endswith('depth')]

In [28]:
subdirs

[]

In [24]:
def add_borders_to_resize(img):
    # Add borders to depth images to make them the same size as the rgb images
    c, r = img.size
    img = np.array(img)
    depth_bigger = np.zeros((r, c, 3), dtype=np.uint8)
    nr, nc = 424, 512
    depthimg = cv2.merge((img,img,img))
    depth_bigger[int(r/2-nr/2):int(r/2+nr/2), int(c/2-nc/2):int(c/2+nc/2)] = depthimg
    return depth_bigger


def crop_image(img, actual_width, actual_height):
    # Crop image to remove black borders
    left_width = (img.width - actual_width) // 2
    right_width = left_width + actual_width

    top_height = (img.height - actual_height) // 2
    bottom_height = top_height + actual_height

    img = img.crop((left_width, top_height, right_width, bottom_height))
    return img
    

def resize_image(img, imgtype='rgb'):
    # Resize RGB images to 1280x720
    # Resize Omni images to 1900x something
    if imgtype == 'rgb': 
        if img.size != (1280, 720):
            img.thumbnail((1280, 720),Image.ANTIALIAS) # Apparently preserves aspect ratio (need to test)
            return img
            # img = img.resize((1280, 720), Image.ANTIALIAS)

    elif imgtype == 'omni':
        if img.size != (1200, 900):
            img = crop_image(img, actual_width=1200, actual_height=900)
            return img
    
    elif imgtype == 'depth' or imgtype == 'ir':
        
        if img.size != (1280, 720):
            img = add_borders_to_resize(img)
            # img = crop_image(img, actual_width=512, actual_height=424)
            return img

    else:
        raise ValueError('Invalid image imgtype')

    return None

In [25]:
def resize_images_in_dir(dirs, imtype):
    for dir in dirs: 
        filetype = dir.split('/')[-1]
        # dst = os.path.join(dir, 'resized')
        dst = '/media/sid/Seagate Expansion Drive/old-recordings/calibration-images/common/calibration_images_common_from_drive/kinect0/stereo-calibration/resized-for-matlab'
        dst = os.path.join(dst, filetype)
        if not os.path.isdir(dst):
            os.mkdir(dst)
        files = [x for x in os.listdir(dir) if x.endswith('.jpg')]
        print(len(files))
        for file in files:
            img = Image.open(os.path.join(dir, file))
            img = resize_image(img, imtype)
            if img is not None:
                img.save(os.path.join(dst, file))

In [26]:
# resize_images_in_dir(rgb_dirs, 'rgb')
# print('resized rgb images')

# resize_images_in_dir(omni_dirs, 'omni')
# print('resized omni images')

resize_images_in_dir(ir_dirs, 'ir')
print('resized ir images')

# resize_images_in_dir(depth_dirs, 'depth')
# print('resized depth images')

resized ir images


In [11]:
src = '/media/sid/Seagate Expansion Drive/calibration-images'
subdirs = [x[0] for x in os.walk(src)]
rgb_dirs = [x for x in subdirs if x.endswith('rgb')]
omni_dirs = [x for x in subdirs if x.endswith('omni')]
ir_dirs = [x for x in subdirs if x.endswith('ir')]
depth_dirs = [x for x in subdirs if x.endswith('depth')]

In [12]:
resize_images_in_dir(rgb_dirs, 'rgb')
print('resized rgb images')
subdirs = [x[0] for x in os.walk(src)]
resize_images_in_dir(ir_dirs, 'ir')
print('resized ir images')

resize_images_in_dir(depth_dirs, 'depth')
print('resized depth images')

resized rgb images
resized omni images
resized ir images
resized depth images


In [3]:
src = '/media/sid/Seagate Expansion Drive/calibration-images'
subdirs = [x[0] for x in os.walk(src)]
omni_dirs = [x for x in subdirs if x.endswith('omni0')] + [x for x in subdirs if x.endswith('omni1')]

In [9]:
resize_images_in_dir(omni_dirs, 'omni')
print('resized omni images')

resized omni images


In [ ]:
# Search for folders in the main directory, 
# check if they have a resized folder,
# if so, check the filenames in the two folders and switch them around

In [ ]:
# Swap between resized and original images
subdirs = [x[0] for x in os.walk(src)]
for i in subdirs:
    if i == 'resized':
        src = '.'
        dst = '../'
        for file in os.listdir(i):